In [146]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import torch.nn.functional as F
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/2024-artificial-intelligence-hw-1/sample_submission.csv
/kaggle/input/2024-artificial-intelligence-hw-1/train.csv
/kaggle/input/2024-artificial-intelligence-hw-1/test.csv


## Define Linear Classifier

In [147]:
class LinearClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Increase number of neurons
        self.fc2 = nn.Linear(128, 64)  # Add an additional hidden layer
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))  # Additional layer with activation
        output = self.fc3(x)
        return output


## Load Data

In [148]:
# load train.csv
train_df = pd.read_csv("/kaggle/input/2024-artificial-intelligence-hw-1/train.csv")
train_df

,date,close,open,high,low,volume,ht_dcperiod,ht_dcphase,inphase,quadrature,...,var,atr,natr,trange,ad,adosc,obv,1_trend,5_trend,10_trend
0,1,94.86,101.36,103.36,94.36,21789208,16.827667,307.142904,-3.555735,-3.246850,...,10.243520,7.920250,8.349410,9.0,-3.848627e+07,-1.167386e+07,261516573.0,1.0,1.0,1.0
1,2,97.59,96.09,99.09,93.09,13642091,15.896730,-22.568827,-2.639682,1.304832,...,7.281680,7.821542,8.014696,6.0,-4.412492e+07,-1.520909e+07,258865213.0,0.0,1.0,1.0
2,3,97.59,94.09,101.59,92.09,16305307,15.562605,-7.225711,-3.147920,-1.168940,...,6.959584,7.941432,8.137547,9.5,-4.155040e+07,-1.235346e+07,258865213.0,1.0,1.0,1.0
3,4,98.69,100.19,102.69,98.69,14127338,15.367623,6.165014,-3.464886,-0.187496,...,3.781264,7.738472,7.841192,5.1,-5.567774e+07,-1.455727e+07,272992551.0,-1.0,0.0,1.0
4,5,96.50,102.00,103.50,95.50,10304396,15.549624,16.710140,-3.691806,-1.918849,...,3.172496,7.757153,8.038501,8.0,-6.340604e+07,-1.659444e+07,262688155.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857,1858,131.64,131.64,133.14,131.64,1915360,21.933646,254.843013,1.760688,-0.225027,...,4.348616,2.505854,1.903565,1.5,-1.378997e+09,-1.715582e+06,853638722.0,1.0,-1.0,-1.0
1858,1859,133.34,132.34,134.84,132.34,2765566,20.412209,305.502984,-0.752763,-6.230444,...,0.751400,2.571374,1.928434,2.5,-1.378417e+09,-1.447268e+06,859048286.0,0.0,-1.0,-1.0
1859,1860,134.19,133.19,134.69,132.69,1102925,20.218289,-37.385040,-3.042672,-4.300574,...,0.751400,2.530561,1.885805,2.0,-1.377866e+09,-1.119953e+06,860151211.0,0.0,-1.0,-1.0
1860,1861,133.77,134.27,136.27,133.27,1224191,20.742574,-29.806261,-3.829639,-1.516603,...,0.839464,2.564093,1.916792,3.0,-1.378682e+09,-1.143915e+06,858927020.0,0.0,-1.0,-1.0


## Preprocess

In [149]:
# CrossEntropyLoss need values start from 0 and can't use negative value
class_transform = {-1:0, 0:1, 1:2}

# Create feature. Per 30 days data as an example.
def create_lagged_features(df, days=30):
    features = []
    targets_1d = []
    targets_5d = []
    targets_10d = []
    
    for i in range(days-1, len(df)):
        feature_row = []
        target_1d = class_transform[df.iloc[i]['1_trend']]
        target_5d = class_transform[df.iloc[i]['5_trend']]
        target_10d = class_transform[df.iloc[i]['10_trend']]

        # Concat features from day 1 to day 30.
        for j in range(days-1, -1, -1):
            feature_row.extend([
                df.iloc[i - j]['close'],
                df.iloc[i - j]['open'],
                df.iloc[i - j]['high'],
                df.iloc[i - j]['low'],
                df.iloc[i - j]['volume']
            ])
        features.append(feature_row)
        targets_1d.append(target_1d)
        targets_5d.append(target_5d)
        targets_10d.append(target_10d)
    
    return np.array(features), np.array(targets_1d), np.array(targets_5d), np.array(targets_10d)

# Create features and targets for 1-day, 5-day, and 10-day trends
features, targets_1d, targets_5d, targets_10d = create_lagged_features(train_df, days=30)

In [150]:
print(features.shape)



(1833, 150)


### Standardization & Split Data

In [151]:
# Define StandardScaler for Standardization
scaler = StandardScaler()

X_train, X_val = features[:round(len(features) * 0.8)], features[round(len(features) * 0.8):]
X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_val = torch.tensor(scaler.transform(X_val), dtype=torch.float32)

y_train_1d = torch.from_numpy((targets_1d[:round(len(targets_1d) * 0.8)]).squeeze()).long()
y_val_1d = torch.from_numpy((targets_1d[round(len(targets_1d) * 0.8):]).squeeze()).long()

y_train_5d = torch.from_numpy((targets_5d[:round(len(targets_5d) * 0.8)]).squeeze()).long()
y_val_5d = torch.from_numpy((targets_5d[round(len(targets_5d) * 0.8):]).squeeze()).long()

y_train_10d = torch.from_numpy((targets_10d[:round(len(targets_10d) * 0.8)]).squeeze()).long()
y_val_10d = torch.from_numpy((targets_10d[round(len(targets_10d) * 0.8):]).squeeze()).long()

# Move data to device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train, X_val = X_train.to(device), X_val.to(device)
y_train_1d, y_val_1d = y_train_1d.to(device), y_val_1d.to(device)
y_train_5d, y_val_5d = y_train_5d.to(device), y_val_5d.to(device)
y_train_10d, y_val_10d = y_train_10d.to(device), y_val_10d.to(device)

In [152]:
# Use DataLoader for each model's dataset
def get_dataloader(X, y):
    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=1024)

train_loader_1d = get_dataloader(X_train, y_train_1d)
val_loader_1d = get_dataloader(X_val, y_val_1d)

train_loader_5d = get_dataloader(X_train, y_train_5d)
val_loader_5d = get_dataloader(X_val, y_val_5d)

train_loader_10d = get_dataloader(X_train, y_train_10d)
val_loader_10d = get_dataloader(X_val, y_val_10d)

## Training

### Define Validation Process

In [153]:
def validate_loss(model):
    model.eval()
    valid_pred = []
    valid_y = []
    for val_batch_X, val_batch_y in val_dataloader:
        # Stop gradient calculation in inference process
        with torch.no_grad():
            val_outputs = model(val_batch_X)
            val_loss = criterion(val_outputs, val_batch_y)
    return val_loss.item()

### Start Training

In [154]:
input_size = 30 * 5  # Each sample has 30 days of 5 features (close, open, high, low, volume)
output_size = 3      # 3 classes for trend prediction

model_1d = LinearClassifier(input_size, output_size).to(device)
model_5d = LinearClassifier(input_size, output_size).to(device)
model_10d = LinearClassifier(input_size, output_size).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_1d = torch.optim.SGD(model_1d.parameters(), lr=1e-4)
optimizer_5d = torch.optim.SGD(model_5d.parameters(), lr=1e-4)
optimizer_10d = torch.optim.SGD(model_10d.parameters(), lr=1e-4)



# Validation function
def validate_loss(model, val_loader):
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for val_batch_X, val_batch_y in val_loader:
            val_outputs = model(val_batch_X)
            val_loss = criterion(val_outputs, val_batch_y)
            total_val_loss += val_loss.item()
    return total_val_loss / len(val_loader)

# Training function for each model
def train_model(model, optimizer, train_loader, val_loader, num_epochs=5000):
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            val_loss = validate_loss(model, val_loader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Train each model
train_model(model_1d, optimizer_1d, train_loader_1d, val_loader_1d)
train_model(model_5d, optimizer_5d, train_loader_5d, val_loader_5d)
train_model(model_10d, optimizer_10d, train_loader_10d, val_loader_10d)

torch.cuda.empty_cache()

Epoch [100/5000], Loss: 1.1011, Val Loss: 1.1041
Epoch [200/5000], Loss: 1.1008, Val Loss: 1.1038
Epoch [300/5000], Loss: 1.1005, Val Loss: 1.1035
Epoch [400/5000], Loss: 1.1003, Val Loss: 1.1032
Epoch [500/5000], Loss: 1.1000, Val Loss: 1.1029
Epoch [600/5000], Loss: 1.0998, Val Loss: 1.1026
Epoch [700/5000], Loss: 1.0996, Val Loss: 1.1023
Epoch [800/5000], Loss: 1.0994, Val Loss: 1.1020
Epoch [900/5000], Loss: 1.0993, Val Loss: 1.1018
Epoch [1000/5000], Loss: 1.0991, Val Loss: 1.1015
Epoch [1100/5000], Loss: 1.0990, Val Loss: 1.1013
Epoch [1200/5000], Loss: 1.0988, Val Loss: 1.1010
Epoch [1300/5000], Loss: 1.0987, Val Loss: 1.1008
Epoch [1400/5000], Loss: 1.0986, Val Loss: 1.1006
Epoch [1500/5000], Loss: 1.0985, Val Loss: 1.1004
Epoch [1600/5000], Loss: 1.0984, Val Loss: 1.1001
Epoch [1700/5000], Loss: 1.0983, Val Loss: 1.0999
Epoch [1800/5000], Loss: 1.0982, Val Loss: 1.0997
Epoch [1900/5000], Loss: 1.0981, Val Loss: 1.0995
Epoch [2000/5000], Loss: 1.0980, Val Loss: 1.0993
Epoch [21

## Testing

In [155]:
test_df = pd.read_csv("/kaggle/input/2024-artificial-intelligence-hw-1/test.csv")
test_df

,id,date,close,open,high,low,volume,ht_dcperiod,ht_dcphase,inphase,...,linearreg_slope,stddev,tsf,var,atr,natr,trange,ad,adosc,obv
0,0,1,143.99,142.99,143.99,141.99,1369798,16.290696,212.093803,1.204988,...,0.134000,0.877460,144.782857,0.769936,3.528314,2.450388,2.00,-1.405847e+09,1.350707e+06,871331963.0
1,0,2,144.42,144.42,144.92,141.92,1196222,16.459398,200.424594,2.559340,...,0.131319,0.808490,144.854176,0.653656,3.490577,2.416963,3.00,-1.405050e+09,1.618916e+06,872528185.0
2,0,3,145.27,145.77,146.27,144.77,1221903,16.608715,205.594921,1.388990,...,0.203341,0.808490,145.425055,0.653656,3.373393,2.322154,1.85,-1.405457e+09,1.451868e+06,873750088.0
3,0,4,152.94,145.44,153.44,144.94,7824953,16.888389,178.143808,-0.724966,...,0.363319,3.294294,147.689890,10.852376,3.615324,2.363884,8.50,-1.397613e+09,3.646685e+06,882514490.0
4,0,5,149.53,151.53,155.03,148.53,4450101,17.143087,167.720081,-1.350238,...,0.461604,3.207036,148.822747,10.285080,3.821372,2.555589,6.50,-1.400694e+09,3.192448e+06,878064389.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,426,26,152.60,149.60,152.60,149.60,2019624,28.715604,208.685202,1.991429,...,-0.153802,2.108867,152.455055,4.447320,4.077149,2.671788,3.00,-1.396747e+09,-1.638342e+06,880974185.0
12806,426,27,154.86,155.36,155.86,154.36,1234617,28.910077,219.954123,-4.560006,...,-0.194593,2.570230,152.544835,6.606080,3.821623,2.467792,1.50,-1.395967e+09,-5.561426e+05,882761400.0
12807,426,28,154.41,152.41,155.41,151.41,1368270,30.321032,221.379706,-8.170982,...,-0.190066,1.908639,152.623077,3.642904,3.834364,2.483236,4.00,-1.395283e+09,-1.754569e+05,881393130.0
12808,426,29,156.21,155.71,158.21,154.71,1809694,31.793577,219.653663,-7.402138,...,-0.136835,1.162986,153.195165,1.352536,3.831910,2.453050,3.80,-1.395541e+09,-8.603005e+04,883202824.0


In [156]:
# Adapted create_test_features function
def create_test_features(df, days=30):
    features = []
    for i in range(days-1, len(df)+1, days):
        feature_row = []
        for j in range(days-1, -1, -1):
            feature_row.extend([
                df.iloc[i - j]['close'],
                df.iloc[i - j]['open'],
                df.iloc[i - j]['high'],
                df.iloc[i - j]['low'],
                df.iloc[i - j]['volume']
            ])
        features.append(feature_row)
    
    return np.array(features)

# Create test features
test_features = create_test_features(test_df, days=30)

# Convert to PyTorch tensors
test_features = torch.from_numpy(test_features).float()

# Standardize the test features
test_features = torch.tensor(scaler.transform(test_features), dtype=torch.float32)
test_features = test_features.to(device)

# Run test features through each model and get predictions
def predict_for_model(model, test_features):
    with torch.no_grad():
        logits = model(test_features)
        probs = F.softmax(logits, dim=1)  # Convert logits to probabilities
        predicted_class = torch.argmax(probs, dim=1)  # Choose class with the highest probability
    return predicted_class.cpu()

# Predictions for each trend
predicted_1d_class = predict_for_model(model_1d, test_features)
predicted_5d_class = predict_for_model(model_5d, test_features)
predicted_10d_class = predict_for_model(model_10d, test_features)

# Print predictions for each trend
print(f'Predicted 1-day class: {predicted_1d_class}')
print(f'Predicted 5-day class: {predicted_5d_class}')
print(f'Predicted 10-day class: {predicted_10d_class}')


Predicted 1-day class: tensor([2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2,
        1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 2, 2, 1, 0, 1, 0, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 2, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 2, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 1, 0, 2, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 2, 0, 2,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 0, 0, 2, 0, 2, 1, 1, 1, 0, 2,
        0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 2, 1, 1,
        0, 0, 0, 

In [157]:
combined_predictions = np.concatenate([predicted_1d_class, predicted_5d_class, predicted_10d_class])

ids = np.arange(0, len(combined_predictions))

results_df = pd.DataFrame({
    'id': ids,
    'trend': combined_predictions
})
results_df['trend'] = results_df['trend'].replace({2: 1, 1: 0, 0: -1})
results_df

,id,trend
0,0,1
1,1,-1
2,2,-1
3,3,0
4,4,0
...,...,...
1276,1276,1
1277,1277,-1
1278,1278,1
1279,1279,1


In [158]:
results_df.to_csv('submission.csv', index=False)